im a coping this notebook https://github.com/evasnow1992/DeepGenerativeModelLINCS/blob/master/deep_generative_model_LINCS.ipynb

In [1]:
import pandas as pd
import gzip, shutil
import os
import pkg_resources

from cmapPy.pandasGEXpress.parse import parse

C:\Users\Usuario\AppData\Local\Temp\ipykernel_24524\2811086252.py:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


## Load GSE70138 small molecular perturbation L1000 data

In [2]:
import pandas as pd
import gzip

# gene info 
file_path = r"C:\Users\Usuario\Desktop\TFM\datos\CLUE_data\CMAPS\GSE70138_Broad_LINCS_gene_info_2017-03-06.txt.gz"
with gzip.open(file_path, 'rt') as f:
    gene_info = pd.read_csv(f, sep='\t')

# cell info 
file_path = r"C:\Users\Usuario\Desktop\TFM\datos\CLUE_data\CMAPS\GSE70138_Broad_LINCS_cell_info_2017-04-28.txt.gz"
with gzip.open(file_path, 'rt') as f:
    cell_info = pd.read_csv(f, sep='\t')

# pert info 
file_path = r"C:\Users\Usuario\Desktop\TFM\datos\CLUE_data\CMAPS\GSE70138_Broad_LINCS_pert_info.txt.gz"
with gzip.open(file_path, 'rt') as f:
    pert_info = pd.read_csv(f, sep='\t')

# inst info 
file_path = r"C:\Users\Usuario\Desktop\TFM\datos\CLUE_data\CMAPS\GSE70138_Broad_LINCS_inst_info_2017-03-06.txt.gz"
with gzip.open(file_path, 'rt') as f:
    inst_info = pd.read_csv(f, sep='\t')

# sig info 
file_path = r"C:\Users\Usuario\Desktop\TFM\datos\CLUE_data\CMAPS\GSE70138_Broad_LINCS_sig_info_2017-03-06.txt.gz"
with gzip.open(file_path, 'rt') as f:
    sig_info = pd.read_csv(f, sep='\t')

# sig metrics 
file_path = r"C:\Users\Usuario\Desktop\TFM\datos\CLUE_data\CMAPS\GSE70138_Broad_LINCS_sig_metrics_2017-03-06.txt.gz"
with gzip.open(file_path, 'rt') as f:
    sig_metrics = pd.read_csv(f, sep='\t')

### Load signature metadata

In [3]:
cellLineNameSM = set(["A375", "HA1E", "HELA", "HT29", "MCF7", "PC3", "YAPC"])
cellLineNameGP = set(["A375", "A549", "HA1E", "HCC515", "HEPG2", "HT29", "MCF7", "PC3", "VCAP"])

In [10]:
sigInfoSM = sig_info
cellLineInfoSM = sigInfoSM[sigInfoSM["cell_id"].isin(cellLineNameSM)]
cellLineSigSM = cellLineInfoSM["sig_id"]
cellLinePertSM = cellLineInfoSM["pert_id"]
cellLinePertTypeSM = cellLineInfoSM["pert_type"]
print(sigInfoSM.shape)
print(cellLineInfoSM.shape)


(118050, 8)
(85183, 8)


In [11]:
pertDicSM = {}
pertArraySM = cellLinePertSM.values
for i in range(len(pertArraySM)):
    pertSM = pertArraySM[i]
    c = pertDicSM.setdefault(pertSM, 0)
    pertDicSM[pertSM] = c + 1
print("The number of perturbagens: " + str(len(pertDicSM)))

The number of perturbagens: 1797


### Load gene information

In [12]:
geneInfoSM = gene_info
lmInfoSM = geneInfoSM["pr_gene_id"][geneInfoSM["pr_is_lm"] == 1].astype(str) # landmark genes only #OJJJJOOOO
print(geneInfoSM.shape)
print(lmInfoSM.shape)

(12328, 5)
(978,)


In [13]:
lmInfoSM

0          780
1         7849
25        6193
43          23
49        9552
         ...  
12184     5467
12223     2767
12224    23038
12286    57048
12321    79716
Name: pr_gene_id, Length: 978, dtype: object

### Load perturbagen class (PCL) information

esto no lo tengo si que  voy a subir el pretraineado con SMP porque esta hecho con las drugs 

# Subimos el modelo pretraineado para SMP (small molleculues and drugs)

In [2]:
#!pip uninstall torch
!pip install torch 

In [3]:
import torch
from torch import nn
from torch.autograd import Variable

# Load model components
encoder = torch.load(r'C:\Users\Usuario\Desktop\TFM\datos\VAE\VAE_encode_SMP.pth')
mu_layer = torch.load(r'C:\Users\Usuario\Desktop\TFM\datos\VAE\VAE_mu_SMP.pth')
logvar_layer = torch.load(r'C:\Users\Usuario\Desktop\TFM\datos\VAE\VAE_logvar_SMP.pth')



ModuleNotFoundError: No module named 'torch._C'

In [ ]:
encoder.eval()
mu_layer.eval()
logvar_layer.eval()

# Your input data (as a tensor, shape: [batch_size, 978])
# Example: input_data = torch.tensor(my_gene_data).float()

with torch.no_grad():
    h1 = encoder[1](encoder[0](input_data))
    h = encoder[3](encoder[2](h1))
    mu = mu_layer(h)
    logvar = logvar_layer(h)
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    z = eps * std + mu  # latent vector

# Now z is your encoded latent vector
